# create a dynamic (self-constructing) chain

The provided guide demonstrates how to create a dynamic chain in LangChain that adjusts itself at runtime. 

# Key Concepts:
1.Dynamic Chains with RunnableLambda:

A RunnableLambda can return another Runnable. When this happens, the new Runnable is executed as part of the overall chain.

This allows constructing parts of the chain dynamically based on the input.

2.Core Components:

RunnablePassthrough: Passes inputs as-is without modification.

chain Decorator: Converts a function into a Runnable.

Custom Runnables: Define logic to decide which chain or logic path to use at runtime.

3. Contextualization Logic:

If the chat_history exists, the contextualize_question chain is executed to reformat the question in context.

Otherwise, the question is passed directly downstream.


In [1]:
pip install -qU langchain-openai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install langchain-anthropic

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-anthropic (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchain-anthropic


# Step 1: Define Prerequisites


In [8]:
import getpass
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o-mini")


# Step 2: Define Components
1. Prompts for Contextualization and QA:

* contextualize_prompt: Reformats the question in context.
* qa_prompt: Generates answers based on retrieved context.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_instructions),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)

contextualize_question = contextualize_prompt | llm | StrOutputParser()

qa_instructions = (
    """Answer the user question given the following context:\n\n{context}."""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_instructions), ("human", "{question}")]
)


2 Dynamic Chain Logic (contextualize_if_needed):

* Uses RunnableLambda to conditionally return contextualize_question if chat_history is present, or passes the question through directly.

In [12]:
from langchain_core.runnables import Runnable, RunnablePassthrough, chain
from operator import itemgetter

@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        return contextualize_question  # Returns Runnable for reformatting
    else:
        return RunnablePassthrough() | itemgetter("question")


3 . Mock Retriever (fake_retriever):

Simulates document retrieval by returning static context for demonstration.

In [13]:
@chain
def fake_retriever(input_: dict) -> str:
    return "egypt's population in 2024 is about 111 million"


# Step 3: Combine Components into the Full Chain
 * Dynamic Chain Construction:

        * Combines the contextualizer, retriever, and QA steps into a complete processing pipeline.




In [14]:
full_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed).assign(
        context=fake_retriever
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)


# Step 4: Invoke and Test
Input example with chat_history:


In [15]:
response = full_chain.invoke(
    {
        "question": "what about egypt",
        "chat_history": [
            ("human", "what's the population of indonesia"),
            ("ai", "about 276 million"),
        ],
    }
)

print(response)


As of 2024, Egypt's population is about 111 million.
